# TO master
特定機材価格を日付に対応したマスターとして作成する。
特定機材価格は改定月の翌月2日から有効なものとなる。

In [1]:
import pandas as pd
import json
import pathlib
import datetime
import numpy as np

ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")
ts

'20231122_150209_109733'

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 250)

マスター読み込み関数  
マスターにはカラム名が無いので、c+番号でカラム名をつける

28, 29, 30 are dates

In [10]:
def get_file_df(filepath):
    if filepath == "/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tensuhyo/kihonmasta/h28/kihonmasta_05.files/t_ALL20160304.csv":
        df_i = pd.read_csv(filepath,encoding='utf-16', header=None,low_memory=False, dtype={27:str, 28:str, 29:str})
    else:
        df_i = pd.read_csv(filepath,encoding='cp932', header=None,low_memory=False, dtype={27:str, 28:str, 29:str})
    df_i.columns = ["c"+ str(i) for i in range(len(df_i.columns))]
    df_i["filepath"] = f
    return df_i

In [11]:
# f_pk = "./masterfile_20230426_150151_224504.pkl"
# f_pk = "./masterfile_20230427_095353_581518.pkl"
# f_pk = "./masterfile_20230603_100705_814763.pkl"
f_pk = "./masterfile_20231122_100722_065215.pkl"
df_files = pd.read_pickle(f_pk)

df_files.head()

,filepath,filename,parent,gparent,bearfile,timeinfo,prefix,ymdinfo,postfix,is_kaitei,ymd,next_month_day1,next_month_day2,sha256,md5,size,lastfmodification,filecreation
0,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,05算定回数テーブル.csv,サンプル,osirase_04,05算定回数テーブル,,05算定回数テーブル,,,False,NaT,NaT,NaT,890542a2abd7d040ab78783bb838af1c456dcdd54e314e...,59b395aa7c467f838d818cd0bf2580cd,10.673828125kB,2023-11-22 10:07:26.207215,2023-11-22 10:07:26.207215
1,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,01補助マスターテーブル.csv,tensuhyo_02,tensuhyo_02_20120213,01補助マスターテーブル,,01補助マスターテーブル,,,False,NaT,NaT,NaT,38cfc25516491e252aef17f93ef4fa4de0ca64ee093485...,cf29e86e3c71970e1f4f88e1e74f5379,886.939453125kB,2023-11-22 10:07:26.234844,2023-11-22 10:07:26.234844
2,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,02包括テーブル.csv,tensuhyo_02,tensuhyo_02_20120213,02包括テーブル,,02包括テーブル,,,False,NaT,NaT,NaT,2b939f604846452eed5c891b4ad97d2b8d9633aa0d9f06...,70d068ccdf6093f9b24b963bb28062e1,6001.912109375kB,2023-11-22 10:07:26.317167,2023-11-22 10:07:26.317167
3,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,03-1背反テーブル1.csv,tensuhyo_02,tensuhyo_02_20120213,03-1背反テーブル1,1,03-1背反テーブル,1,,False,NaT,NaT,NaT,e9abb2b236605c230435dafe08e09bbf93941b402582a8...,9443610957dd09433753eb9230b2e1bf,578.05078125kB,2023-11-22 10:07:26.325180,2023-11-22 10:07:26.325180
4,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,03-2背反テーブル2.csv,tensuhyo_02,tensuhyo_02_20120213,03-2背反テーブル2,2,03-2背反テーブル,2,,False,NaT,NaT,NaT,92000a77113ec6044414d4f6a325a44463d850db653205...,fcb7bb8a12b2244dc42dffb0ed5ad644,392.81640625kB,2023-11-22 10:07:26.332179,2023-11-22 10:07:26.332179


TOマスターのみ

In [12]:
df_files_ =  df_files\
    .query("prefix == 't' or prefix == 't_ALL' ")\
    .drop_duplicates(subset=["filename", "sha256", "md5", "size"], keep='first', ) \
    .reset_index(drop=False)

In [13]:
df_files_.head().T

,0,1,2,3,4
index,264,265,266,267,268
filepath,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...
filename,t20120305.csv,t20120319.csv,t20120629.csv,t20120907.csv,t20120928.csv
parent,kihonmasta_05.files,kihonmasta_05.files,kihonmasta_05.files,kihonmasta_05.files,kihonmasta_05.files
gparent,h24,h24,h24,h24,h24
bearfile,t20120305,t20120319,t20120629,t20120907,t20120928
timeinfo,20120305,20120319,20120629,20120907,20120928
prefix,t,t,t,t,t
ymdinfo,20120305,20120319,20120629,20120907,20120928
postfix,,,,,


In [7]:
# df_files_["prefix"].count()

df_dt from step 2

In [14]:
# fdt_pk = "./datefile_20230426_100605_859194.pkl"
# fdt_pk = "./datefile_20230427_095353_581518.pkl"
# fdt_pk = "./datefile_20230603_100705_814763.pkl"
fdt_pk = "./datefile_20231122_100722_065215.pkl"
dfdt = pd.read_pickle(fdt_pk)
dfdt.head()

,dt,dt_str,yy_str,mm_str,dd_str,is_past
0,2010-01-01,2010-01-01,2010,1,1,True
1,2010-01-02,2010-01-02,2010,1,2,True
2,2010-01-03,2010-01-03,2010,1,3,True
3,2010-01-04,2010-01-04,2010,1,4,True
4,2010-01-05,2010-01-05,2010,1,5,True


In [15]:
dt_start = dfdt["dt"].iat[0]
dt_start_str = dfdt["dt_str"].iat[0]
dt_start, dt_start_str

(Timestamp('2010-01-01 00:00:00'), '2010-01-01')

In [16]:
dt_end = dfdt["dt"].iat[-1]
dt_end_str = dfdt["dt_str"].iat[-1]
dt_end,dt_end_str

(Timestamp('2023-08-31 00:00:00'), '2023-08-31')

当該ファイル全部読み込み

In [17]:
%%time

l_df = []
for r in df_files_.itertuples():
    # print(r)
    f = r.filepath
    #print(f)
    l_df.append(get_file_df(f))
# df_i.head()

CPU times: user 1.07 s, sys: 357 ms, total: 1.43 s
Wall time: 2.53 s


In [18]:
def pick_dt_to(c_change, c_pend, c_enduse):
    l_res = []
    for s_change, s_pend,  s_enduse in zip(c_change, c_pend, c_enduse):
        if s_change != "":
            l_res.append(s_change)
        elif s_pend !="":
            l_res.append(s_pend)
        elif s_enduse:
            l_res.append(s_enduse)
        else:
            l_res.append("")
    return l_res

In [43]:
df_whole = pd.concat(l_df)\
    .merge(df_files_, on="filepath") \
    .assign(c27 = lambda df:df.c27.fillna(""),
            c28 = lambda df:df.c28.fillna("").str.replace("00000000", ""),
            c29 = lambda df:df.c29.fillna("").str.replace("99999999", "")
           )\
    .assign(ymdc = lambda df: pick_dt_to(df.c27, df.c28, df.c29))\
    .assign(ymdinfo = lambda df: [r.ymdc if r.ymdc != "" else r.ymd for r in df.itertuples()]) \
    .assign(ymd = lambda df:pd.to_datetime(df.ymdinfo, errors="coerce"))\
    .sort_values(["c2", "ymd"])\
    .assign(next_month_day1 = lambda df:df.ymd + pd.offsets.MonthBegin(1)) \
    .assign(next_month_day2 = lambda df:df.next_month_day1 + datetime.timedelta(days =1))
    # .sort_values(["c2", "ymdinfo"])
df_whole.shape

(21978, 57)

In [44]:
df_whole.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19,c20,c21,c22,c23,c24,...,c32,c33,c34,c35,c36,filepath,index,filename,parent,gparent,bearfile,timeinfo,prefix,ymdinfo,postfix,is_kaitei,ymd,next_month_day1,next_month_day2,sha256,md5,size,lastfmodification,filecreation,ymdc
0,0,T,700010000,2.0,半切,4.0,ﾊﾝｾﾂ,6.0,1.0,枚,1.0,137.0,0.0,1.0,0.0,0.0,1.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,NaN,NaN,NaN,フィルム・半切,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,264,t20120305.csv,kihonmasta_05.files,h24,t20120305,20120305,t,20120401,,False,2012-04-01,2012-05-01,2012-05-02,26c366990e6157d46f5533956609d48d127ad6e8a3e387...,97ca7744f169319c2833be3661cb2449,292.8955078125kB,2016-02-04 20:29:48,2023-11-22 09:57:54.804966,20120401
1198,0,T,700010000,2.0,半切,4.0,ﾊﾝｾﾂ,6.0,1.0,枚,1.0,137.0,0.0,1.0,0.0,0.0,1.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,NaN,NaN,NaN,フィルム・半切,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,275,t_ALL20131227.csv,t_ALL20131227,kihonmasta_05.files,t_ALL20131227,20131227,t_ALL,20120401,,False,2012-04-01,2012-05-01,2012-05-02,fb3869955c7059c69d3262d26ad4443caf85850ea6a908...,32abefc4e0873511e2b5ee24167ec79d,316.0458984375kB,2023-11-22 10:07:36.633899,2023-11-22 10:07:36.633899,20120401
2505,5,T,700010000,2.0,半切,4.0,ﾊﾝｾﾂ,6.0,1.0,枚,1.0,139.0,0.0,1.0,0.0,0.0,1.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,NaN,NaN,NaN,フィルム・半切,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,444,t_ALL20140305.csv,kihonmasta_05.files,h26,t_ALL20140305,20140305,t_ALL,20140401,,False,2014-04-01,2014-05-01,2014-05-02,a24fa20a6e519143ab7743c4b2307bebea4c1774d3332e...,1868fc6e2283631e832212daec4d9edc,336.69921875kB,2016-02-04 17:24:18,2023-11-22 09:57:28.967456,20140401
3655,0,T,700010000,2.0,半切,4.0,ﾊﾝｾﾂ,6.0,1.0,枚,1.0,139.0,0.0,1.0,0.0,0.0,1.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,NaN,NaN,NaN,フィルム・半切,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,445,t_ALL20151228.csv,t_ALL20151228,kihonmasta_05.files,t_ALL20151228,20151228,t_ALL,20140401,,False,2014-04-01,2014-05-01,2014-05-02,a47a64e84cf52f8106e0dacde6791fe9b892b169e5b638...,a0245a922ab5ab78bbf9f39866ee11e2,355.8662109375kB,2023-11-22 10:07:37.507319,2023-11-22 10:07:37.507319,20140401
4951,5,T,700010000,2.0,半切,4.0,ﾊﾝｾﾂ,6.0,1.0,枚,1.0,122.0,0.0,1.0,0.0,0.0,1.0,139.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,NaN,NaN,NaN,フィルム・半切,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,601,t_ALL20160304.csv,kihonmasta_05.files,h28,t_ALL20160304,20160304,t_ALL,20160401,,False,2016-04-01,2016-05-01,2016-05-02,7a949b7cd56a938137dd643c07668eb0dd24f0543758f1...,23ade1e409a7a0f8d960f566239cb1be,650.291015625kB,2016-03-24 15:08:07,2023-11-22 09:57:04.588155,20160401


c4には診療行為省略名称

In [45]:
df_whole\
    .query("c4 == c4").head().T.head(15)

,0,1198,2505,3655,4951
c0,0,0,5,0,5
c1,T,T,T,T,T
c2,700010000,700010000,700010000,700010000,700010000
c3,2.0,2.0,2.0,2.0,2.0
c4,半切,半切,半切,半切,半切
c5,4.0,4.0,4.0,4.0,4.0
c6,ﾊﾝｾﾂ,ﾊﾝｾﾂ,ﾊﾝｾﾂ,ﾊﾝｾﾂ,ﾊﾝｾﾂ
c7,6.0,6.0,6.0,6.0,6.0
c8,1.0,1.0,1.0,1.0,1.0
c9,枚,枚,枚,枚,枚


In [46]:
df_whole\
    .query("c4 == c4")\
    ["c2"].unique().size
    

1553

c10の意味はレセプト電算処理システムマスターファイル使用説明書を参照されたい。  
０：材料価格基準改定又はそれ以降に新設された特定器材（多分）  
１：金額  
２：購入価格  
４：金額（整数部のみ）  
５：％加算  
９：乗算割合


In [47]:
df_whole \
    .query("c10 == 0")\
    .sort_values("timeinfo").T

,568,570,502,501,380,2975,5547,5546,5527,5511,5510,5509,5508,5805,5506,5503,5504,5502,5467,5465,5405,5363,5362,5358,5357,...,19455,19446,19445,19379,19371,19524,20148,19863,19867,20097,20012,19946,19944,19942,19940,19865,19938,19894,19892,19890,19888,19871,19869,19896,20532
c0,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
c1,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T
c2,730770000,730790000,728810000,728800000,710010620,710010711,710010852,710010851,710010832,710010816,710010815,710010814,710010813,730760000,710010811,710010808,710010809,710010807,710010772,710010770,710010697,710010653,710010652,710010648,710010647,...,710010109,710010098,710010097,710010013,710010003,710010187,730710000,710010899,710010903,728910000,710011068,710010990,710010988,710010986,710010984,710010901,710010982,710010934,710010932,710010930,710010928,710010907,710010905,710010936,738760000
c3,32.0,32.0,32.0,32.0,13.0,21.0,18.0,27.0,25.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,25.0,24.0,21.0,32.0,32.0,32.0,32.0,...,20.0,32.0,23.0,23.0,32.0,19.0,15.0,28.0,28.0,28.0,18.0,27.0,28.0,28.0,28.0,28.0,28.0,24.0,24.0,24.0,24.0,27.0,28.0,23.0,28.0
c4,ホローファイバー型及び積層型（キール型）１．５ｍ２未満２（在宅）,ホローファイバー型及び積層型（キール型）１．５ｍ２以上２（在宅）,ホローファイバー型及び積層型（キール型）（１．５ｍ２以上）（２）,ホローファイバー型及び積層型（キール型）（１．５ｍ２未満）（２）,骨セメント（脊椎用（２））,人工股関節用材（骨盤側材・ライナー（６））,人工肘関節用材（上腕骨側材・特殊型）,人工肩関節用材（リバース型・関節窩ヘッド・下方補正型）,人工肩関節用材（リバース型・上腕骨ステム・特殊型）,ホローファイバー型及び積層型（キール型）２．０ｍ２以上５（在宅）,ホローファイバー型及び積層型（キール型）２．０ｍ２以上４（在宅）,ホローファイバー型及び積層型（キール型）１．５ｍ２以上５（在宅）,ホローファイバー型及び積層型（キール型）１．５ｍ２以上４（在宅）,ホローファイバー型・積層型（キール型）１．５ｍ２未満１・２・在宅,ホローファイバー型・積層型（キール型）１．５ｍ２以上１・２・在宅,ホローファイバー型及び積層型（キール型）１．５ｍ２未満３（在宅）,ホローファイバー型及び積層型（キール型）１．５ｍ２未満４（在宅）,ホローファイバー型・積層型（キール型）１．５ｍ２未満１・２・在宅,人工肩関節用材（リバース型・上腕骨ステム・標準型）,オプション部品（人工肩関節再置換用ステムヘッド）,人工股関節用材（骨盤側材・ライナー（５））,ホローファイバー型及び積層型（キール型）（２．０ｍ２以上）（５）,ホローファイバー型及び積層型（キール型）（２．０ｍ２以上）（４）,ホローファイバー型及び積層型（キール型）２．０ｍ２以上５（在宅）,ホローファイバー型及び積層型（キール型）２．０ｍ２以上４（在宅）,...,人工骨（汎用型・非吸収型・骨形成促進型）,プラスチックカニューレ型静脈内留置針（針刺し事故防止機構付加型）,プラスチックカニューレ型静脈内留置針（標準型）,ペースメーカー（トリプルチャンバ１型・標準型）,プラスチックカニューレ型静脈内留置針（針刺し事故防止機構付加型）,歯科鋳造用ニッケルクロム合金鉤・バー用,抗悪性腫瘍剤注入用肝動脈塞栓材,ダイアライザー（１ａ型（膜面積１．５ｍ２未満））（在宅）,ダイアライザー（２ａ型（膜面積１．５ｍ２未満））（在宅）,固定用内副子（スクリュー）（一般（アルミナセラミック））,生体弁（異種心膜弁（２））・指定番号,ダイアライザー（Ｓ型（膜面積１．５ｍ２未満））（在宅）,ダイアライザー（２ｂ型（膜面積１．５ｍ２未満））（在宅）,ダイアライザー（２ａ型（膜面積１．５ｍ２未満））（在宅）,ダイアライザー（１ｂ型（膜面積１．５ｍ２未満））（在宅）,ダイアライザー（１ｂ型（膜面積１．５ｍ２未満））（在宅）,ダイアライザー（１ａ型（膜面積１．５ｍ２未満））（在宅）,ダイアライザー（２ｂ型（膜面積１．５ｍ２未満））,ダイアライザー（２ａ型（膜面積１．５ｍ２未満））,ダイアライザー（１ｂ型（膜面積１．５ｍ２未満））,ダイアライザー（１ａ型（膜面積１．５ｍ２未満））,ダイアライザー（Ｓ型（膜面積１．５ｍ２未満））（在宅）,ダイアライザー（２ｂ型（膜面積１．５ｍ２未満））（在宅）,ダイアライザー（Ｓ型（膜面積１．５ｍ２未満））,人工指関節用材（手指関節用・その他手指関節用・手根骨用）
c5,20.0,20.0,20.0,20.0,6.0,15.0,17.0,16.0,16.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,16.0,10.0,15.0,20.0,20.0,20.0,20.0,...,7.0,20.0,20.0,8.0,20.0,20.0,20.0,9.0,9.0,15.0,7.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,15.0
c6,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ｺﾂｾﾒﾝﾄ,ｼﾞﾝｺｳｺｶﾝｾﾂﾖｳｻﾞｲ,ｼﾞﾝｺｳﾁｭｳｶﾝｾﾂﾖｳｻﾞｲ,ｼﾞﾝｺｳｶﾀｶﾝｾﾂﾖｳｻﾞｲ,ｼﾞﾝｺｳｶﾀｶﾝｾﾂﾖｳｻﾞｲ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ｼﾞﾝｺｳｶﾀｶﾝｾﾂﾖｳｻﾞｲ,ｵﾌﾟｼｮﾝﾌﾞﾋﾝ,ｼﾞﾝｺｳｺｶﾝｾﾂﾖｳｻﾞｲ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,ﾎﾛｰﾌｧｲﾊﾞｰｶﾞﾀｵﾖﾋﾞｾｷｿｳ,...,ｼﾞﾝｺｳｺﾂ,ﾌﾟﾗｽﾁｯｸｶﾆｭｰﾚｶﾞﾀｼﾞｮｳﾐ,ﾌﾟﾗｽﾁｯｸｶﾆｭｰﾚｶﾞﾀｼﾞｮｳﾐ,ﾍﾟｰｽﾒｰｶｰ,ﾌﾟﾗｽﾁｯｸｶﾆｭｰﾚｶﾞﾀｼﾞｮｳﾐ,ｼｶﾁｭｳｿﾞｳﾖｳﾆｯｹﾙｸﾛﾑｺﾞｳ,ｺｳｱｸｾｲｼｭﾖｳｻﾞｲﾁｭｳﾆｭｳﾖ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ｺﾃｲﾖｳﾅｲﾌｸｼｽｸﾘｭｰ,ｾｲﾀｲﾍﾞﾝ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ﾀﾞｲｱﾗｲｻﾞｰ,ｼﾞﾝｺｳｼｶﾝｾﾂﾖｳｻﾞｲ
c7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [48]:
df_whole \
    .groupby(["c10"])\
    ["c2"].count()

c10
0.0      198
1.0    20886
2.0      291
4.0       34
5.0       34
9.0       17
Name: c2, dtype: int64

In [49]:
df_whole \
    .groupby(["c12"])\
    ["c2"]\
    .count()\
    .reset_index()\
    .set_index("c12")

,c2
c12,
0.0,21455
1.0,5


In [50]:
df_whole\
    .groupby("c2")\
    ["c0"].count()\
    .reset_index() \
    .sort_values("c0", ascending = False)

,c2,c0
232,710010176,33
236,710010180,28
230,710010174,27
237,710010181,27
229,710010173,26
...,...,...
385,710010620,1
920,710011157,1
1017,728800000,1
1018,728810000,1


In [51]:
df_whole\
    .groupby("c0")\
    ["c2"].count()\
    .reset_index() 
# \
#     .sort_values("c0", ascending = False)

,c0,c2
0,0,15050
1,3,684
2,5,5966
3,9,278


IYとの違いは
"c23", "c24"→,"c16", "c17"  
 "c20"　酸素区分  
 "c21" 特定機材種類



In [52]:
# df_whole \
#     .sort_values(["c2", "ymd"]) \
#     .query("c2 == 111000370").T

特定機材についてはc10で掛けたりはしていない。
c10=pointlabel=点数識別を付与しておくので、目的に応じて後で処理すること。

In [53]:
def c10c11fix(c10, c11):
    if c10 != c10 or c11 != c11:
        # c10 or c11がNAのならC11はそのまま
        return c11
    elif c11 ==0:
        return c11
    else:
        if c10 in [1,3,4,5,6,7,8]:
            mul = 1
        # elif c10 == 3:
        #     mul = 10
        # elif c10 == 4:
        #     mul = 0
        # elif c10 == 5:
        #     mul = 1
        # elif c10 ==6:
        #     mul = -1
        # elif c10 ==7:
        #     mul = -1
        # elif c10==8:
        #     mul = -10
        else:
            #rint(c10, c11)
            mul=1
        return mul*c11

In [54]:
# .query("c10 != 7 and c10 != 3") \していない。点数もマスターとして作成するため
# c11fillがNAのものは、ようするに2010年マスターを引き継いでいて点数などがわからない。除外する。
# s1>s2も除外する。これは直近のマスターで情報があった項目で、翌月2日よりまえにdt_endが存在する場合である。
df_sub = df_whole\
    .loc[:, ["c0", "c2", "c4", "c9", "c10", "c11", "c16","c17",
             "c20", "c21",
             "ymd", "next_month_day1", "next_month_day2", "filename"]].copy()\
    .sort_values(["c2", "ymd"]) \
    .assign(c11fill_ = lambda df:df.groupby("c2")["c11"].fillna(method="ffill"))\
    .assign(c11fill  = lambda df:[c10c11fix(r.c10, r.c11fill_) for r in df.itertuples()])\
    .assign(c17fill_ = lambda df:df.groupby("c2")["c17"].fillna(method="ffill"))\
    .assign(c17fill  = lambda df:[c10c11fix(r.c16, r.c17fill_) for r in df.itertuples()])\
    .assign(s0 = lambda df:df.groupby("c2")["next_month_day2"].shift(1))\
    .assign(s1 = lambda df:df.next_month_day2)\
    .assign(s2 = lambda df:df.groupby("c2")["next_month_day2"].shift(-1))\
    .assign(dt_rank = lambda df:df.groupby("c2")["ymd"].rank(method='min'))\
    .assign(is_first_in_master = lambda df:np.isnan(df.s0))\
    .assign(is_last_in_master = lambda df:np.isnan(df.s2))\
    .assign(s0 = lambda df:df.s0.fillna(dt_start))\
    .assign(s2 = lambda df:df.s2.fillna(dt_end))\
    .query("s1 <= s2")
df_sub.head(5)

/tmp/ipykernel_69704/604457440.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .assign(c11fill_ = lambda df:df.groupby("c2")["c11"].fillna(method="ffill"))\
/tmp/ipykernel_69704/604457440.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .assign(c17fill_ = lambda df:df.groupby("c2")["c17"].fillna(method="ffill"))\


,c0,c2,c4,c9,c10,c11,c16,c17,c20,c21,ymd,next_month_day1,next_month_day2,filename,c11fill_,c11fill,c17fill_,c17fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master
0,0,700010000,半切,枚,1.0,137.0,1.0,137.0,0.0,0.0,2012-04-01,2012-05-01,2012-05-02,t20120305.csv,137.0,137.0,137.0,137.0,2010-01-01,2012-05-02,2012-05-02,1.0,True,False
1198,0,700010000,半切,枚,1.0,137.0,1.0,137.0,0.0,0.0,2012-04-01,2012-05-01,2012-05-02,t_ALL20131227.csv,137.0,137.0,137.0,137.0,2012-05-02,2012-05-02,2014-05-02,1.0,False,False
2505,5,700010000,半切,枚,1.0,139.0,1.0,137.0,0.0,0.0,2014-04-01,2014-05-01,2014-05-02,t_ALL20140305.csv,139.0,139.0,137.0,137.0,2012-05-02,2014-05-02,2014-05-02,3.0,False,False
3655,0,700010000,半切,枚,1.0,139.0,1.0,137.0,0.0,0.0,2014-04-01,2014-05-01,2014-05-02,t_ALL20151228.csv,139.0,139.0,137.0,137.0,2014-05-02,2014-05-02,2016-05-02,3.0,False,False
4951,5,700010000,半切,枚,1.0,122.0,1.0,139.0,0.0,0.0,2016-04-01,2016-05-01,2016-05-02,t_ALL20160304.csv,122.0,122.0,139.0,139.0,2014-05-02,2016-05-02,2016-05-02,5.0,False,False


In [55]:
df_sub.query("s1 > s2")

,c0,c2,c4,c9,c10,c11,c16,c17,c20,c21,ymd,next_month_day1,next_month_day2,filename,c11fill_,c11fill,c17fill_,c17fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master


## bsl: Before SaLe  
上市前。診療行為を上市というのはおかしいが、ともかく新規にマスターとして登録される前の期間。DB上、上市前に当該コードが存在するかは不明である。なので最も古い価格を与える。
## bdm: Before Degital Master  
最古のデジタルマスターより前。デジタルマスターではそれ以前の値は不明である。なので最も古い価格を与える。  
## ws: Within Sale  
マスターに採用されている期間中。最新の価格改定は翌月２日に価格される。  
## aes: After End of Sale
廃止終了後。DB上、廃止後に当該コードが存在するかは不明である。なので最も新しい価格を与える。

In [56]:
def get_1st_tag (c0):
    if c0 == 3:
        return "bsl"
    elif c0 == 5:
        return "bdm"
    elif c0 == 0:
        return "bdm"
    elif c0 == 9:
        return "bdm"
    else:
        return "unknown"

c0の意味は  
０：前マスターの内容と同じであることを表す。  
１：抹消  
３：新規  
５：変更  
９：廃止  


In [57]:
d_c10 = df_sub \
    .query("c10 == c10")\
    .groupby(["c2"])\
    ["c10"].unique().to_dict()

for k, v in d_c10.items():
    if len(v) != 1:
        print(k)

710010002
710010003
710010005
710010008
710010013
710010017
710010024
710010029
710010030
710010031
710010032
710010033
710010034
710010036
710010037
710010038
710010039
710010040
710010041
710010045
710010078
710010094
710010097
710010098
710010100
710010102
710010109
710010116
710010171
710010177
710010178
710010179
710010184
710010185
710010186
710010187
710010188
710010189
710010195
710010196
710010199
710010200
710010248
710010249
710010290
710010291
710010400
710010552
710010600
710010624
710010631
710010632
710010636
710010637
710010638
710010647
710010648
710010652
710010653
710010686
710010692
710010697
710010711
710010715
710010723
710010735
710010737
710010740
710010743
710010748
710010750
710010751
710010752
710010768
710010770
710010772
710010788
710010807
710010808
710010809
710010810
710010811
710010812
710010813
710010814
710010815
710010816
710010829
710010832
710010844
710010845
710010849
710010851
710010852
710010858
710010861
710010863
710010865
710010867
710010869


In [58]:
# d_c10

In [59]:
df_sub\
    .query("c10 == c10")\
    .groupby("c10")\
    ["c2"].count()

c10
0.0      198
1.0    20872
2.0      291
4.0       34
5.0       34
9.0       17
Name: c2, dtype: int64

上記から、c2にたいしてc10は一意に規定されている様子  
また殆どは金額。


１：金額  
３：点数（プラス）  
４：購入価格（点数）  
５：％加算  
６：％減算  
７：減点診療行為  
８：点数（マイナス）  

新規。上市開始。

In [60]:
df_sub0 = df_sub.query("dt_rank == 1 and c0 ==3")\
    .rename(columns={
    "c10":"pointlab",
    "c11":"point",
    "s0":"t1",
    "s1":"t2"
}) \
    .loc[:,["c0", "c2","t1", "t2","pointlab", "point", "ymd"]]\
    .assign(tag = "bsl",
            ind = 0)
df_sub0.head(5)

,c0,c2,t1,t2,pointlab,point,ymd,tag,ind
7842,3,710010290,2010-01-01,2018-05-02,1.0,11100.0,2018-04-01,bsl,0
7843,3,710010291,2010-01-01,2018-05-02,1.0,18200.0,2018-04-01,bsl,0
15387,3,710010292,2010-01-01,2023-01-02,1.0,48.0,2022-12-01,bsl,0
7844,3,710010294,2010-01-01,2018-05-02,1.0,453.0,2018-04-01,bsl,0
7845,3,710010295,2010-01-01,2018-05-02,1.0,453.0,2018-04-01,bsl,0


初出だが新規ではない。  
医薬品では「前回の価格情報」を利用しているが、得適材はこれが手に入らないので、bdmかつc11fillがここに入る。

In [61]:
df_sub1 = df_sub\
    .query("dt_rank == 1 and c0 !=3")\
    .rename(columns={
    "c10":"pointlab",
    "c11fill":"point",
    "s0":"t1",
    "s1":"t2"
})  \
   .loc[:,["c0", "c2","t1", "t2","pointlab", "point", "ymd"]]\
    .assign(tag = "bdm",
            ind = 0)
df_sub1.head(5)

,c0,c2,t1,t2,pointlab,point,ymd,tag,ind
0,0,700010000,2010-01-01,2012-05-02,1.0,137.0,2012-04-01,bdm,0
1198,0,700010000,2012-05-02,2012-05-02,1.0,137.0,2012-04-01,bdm,0
1,0,700020000,2010-01-01,2012-05-02,1.0,137.0,2012-04-01,bdm,0
1199,0,700020000,2012-05-02,2012-05-02,1.0,137.0,2012-04-01,bdm,0
2,0,700030000,2010-01-01,2012-05-02,1.0,114.0,2012-04-01,bdm,0


終了ではない

In [62]:
df_sub2 = df_sub \
    .query("c0 != 9")\
    .rename(columns={
    "c10":"pointlab",
    "c11fill":"point",
    "s1":"t1",
    "s2":"t2"
}) \
    .loc[:,["c0","c2","t1", "t2","pointlab", "point", "ymd"]] \
    .assign(tag = "ws",
            ind = 1)
df_sub2.head()

,c0,c2,t1,t2,pointlab,point,ymd,tag,ind
0,0,700010000,2012-05-02,2012-05-02,1.0,137.0,2012-04-01,ws,1
1198,0,700010000,2012-05-02,2014-05-02,1.0,137.0,2012-04-01,ws,1
2505,5,700010000,2014-05-02,2014-05-02,1.0,139.0,2014-04-01,ws,1
3655,0,700010000,2014-05-02,2016-05-02,1.0,139.0,2014-04-01,ws,1
4951,5,700010000,2016-05-02,2016-05-02,1.0,122.0,2016-04-01,ws,1


廃止

In [63]:
df_sub3 = df_sub \
    .query("c0 == 9")\
    .rename(columns={
    "c11fill":"point",
    "c10":"pointlab",
    "s1":"t1",
    "s2":"t2"
}) \
    .loc[:,["c0","c2","t1", "t2", "point", "ymd"]] \
    .assign(tag = "aes", ind = 2)
df_sub3.head()

,c0,c2,t1,t2,point,ymd,tag,ind
19370,9,710010002,2020-05-02,2023-08-31,0.0,2020-04-01,aes,2
19371,9,710010003,2020-05-02,2023-08-31,0.0,2020-04-01,aes,2
7579,9,710010005,2018-05-02,2023-08-31,0.0,2018-04-01,aes,2
5018,9,710010008,2016-05-02,2023-08-31,0.0,2016-04-01,aes,2
19379,9,710010013,2020-05-02,2023-08-31,0.0,2020-04-01,aes,2


In [64]:
df_sub\
    .query("c2 ==710010002 ")

,c0,c2,c4,c9,c10,c11,c16,c17,c20,c21,ymd,next_month_day1,next_month_day2,filename,c11fill_,c11fill,c17fill_,c17fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master
61,5,710010002,プラスチックカニューレ型静脈内留置針（標準型）,NaN,1.0,87.0,1.0,90.0,0.0,0.0,2012-04-01,2012-05-01,2012-05-02,t20120305.csv,87.0,87.0,90.0,90.0,2010-01-01,2012-05-02,2012-05-02,1.0,True,False
1259,0,710010002,プラスチックカニューレ型静脈内留置針（標準型）,NaN,1.0,87.0,1.0,90.0,0.0,0.0,2012-04-01,2012-05-01,2012-05-02,t_ALL20131227.csv,87.0,87.0,90.0,90.0,2012-05-02,2012-05-02,2014-05-02,1.0,False,False
2566,5,710010002,プラスチックカニューレ型静脈内留置針（標準型）,NaN,1.0,89.0,1.0,87.0,0.0,0.0,2014-04-01,2014-05-01,2014-05-02,t_ALL20140305.csv,89.0,89.0,87.0,87.0,2012-05-02,2014-05-02,2014-05-02,3.0,False,False
3716,0,710010002,プラスチックカニューレ型静脈内留置針（標準型）,NaN,1.0,89.0,1.0,87.0,0.0,0.0,2014-04-01,2014-05-01,2014-05-02,t_ALL20151228.csv,89.0,89.0,87.0,87.0,2014-05-02,2014-05-02,2016-05-02,3.0,False,False
5012,0,710010002,プラスチックカニューレ型静脈内留置針（標準型）,NaN,1.0,89.0,1.0,89.0,0.0,0.0,2016-04-01,2016-05-01,2016-05-02,t_ALL20160304.csv,89.0,89.0,89.0,89.0,2014-05-02,2016-05-02,2016-05-02,5.0,False,False
6314,0,710010002,プラスチックカニューレ型静脈内留置針（標準型）,NaN,1.0,89.0,1.0,89.0,0.0,0.0,2016-04-01,2016-05-01,2016-05-02,t_ALL20171130.csv,89.0,89.0,89.0,89.0,2016-05-02,2016-05-02,2018-05-02,5.0,False,False
7576,0,710010002,プラスチックカニューレ型静脈内留置針（標準型）,NaN,1.0,89.0,1.0,89.0,0.0,0.0,2018-04-01,2018-05-01,2018-05-02,t20180305.csv,89.0,89.0,89.0,89.0,2016-05-02,2018-05-02,2018-05-02,7.0,False,False
8945,0,710010002,プラスチックカニューレ型静脈内留置針（標準型）,NaN,1.0,89.0,1.0,89.0,0.0,0.0,2018-04-01,2018-05-01,2018-05-02,t_ALL20190830.csv,89.0,89.0,89.0,89.0,2018-05-02,2018-05-02,2019-11-02,7.0,False,False
16857,5,710010002,プラスチックカニューレ型静脈内留置針（標準型）,NaN,1.0,91.0,1.0,89.0,0.0,0.0,2019-10-01,2019-11-01,2019-11-02,t20190819.csv,91.0,91.0,89.0,89.0,2018-05-02,2019-11-02,2019-11-02,9.0,False,False
18120,0,710010002,プラスチックカニューレ型静脈内留置針（標準型）,NaN,1.0,91.0,1.0,89.0,0.0,0.0,2019-10-01,2019-11-01,2019-11-02,t_ALL20191129.csv,91.0,91.0,89.0,89.0,2019-11-02,2019-11-02,2020-05-02,9.0,False,False


In [66]:
df_span = pd.concat([df_sub0, df_sub1, df_sub2, df_sub3])\
    .sort_values(["c2", "ymd", "ind"]).query("t1 != t2")
df_span.head(50)

,c0,c2,t1,t2,pointlab,point,ymd,tag,ind
0,0,700010000,2010-01-01,2012-05-02,1.0,137.0,2012-04-01,bdm,0
1198,0,700010000,2012-05-02,2014-05-02,1.0,137.0,2012-04-01,ws,1
3655,0,700010000,2014-05-02,2016-05-02,1.0,139.0,2014-04-01,ws,1
6253,0,700010000,2016-05-02,2018-05-02,1.0,122.0,2016-04-01,ws,1
8884,0,700010000,2018-05-02,2019-11-02,1.0,122.0,2018-04-01,ws,1
18059,0,700010000,2019-11-02,2020-05-02,1.0,124.0,2019-10-01,ws,1
20695,0,700010000,2020-05-02,2022-05-02,1.0,120.0,2020-04-01,ws,1
15479,0,700010000,2022-05-02,2023-08-31,1.0,120.0,2022-04-01,ws,1
1,0,700020000,2010-01-01,2012-05-02,1.0,137.0,2012-04-01,bdm,0
1199,0,700020000,2012-05-02,2014-05-02,1.0,137.0,2012-04-01,ws,1


In [67]:
dfdt.head()

,dt,dt_str,yy_str,mm_str,dd_str,is_past
0,2010-01-01,2010-01-01,2010,1,1,True
1,2010-01-02,2010-01-02,2010,1,2,True
2,2010-01-03,2010-01-03,2010,1,3,True
3,2010-01-04,2010-01-04,2010,1,4,True
4,2010-01-05,2010-01-05,2010,1,5,True


この処理が重い。全く美しくない。

In [68]:
# df_whole.groupby("c2")["ymd"].min().to_list()

In [69]:
df_whole.columns

Index(['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10',
       'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20',
       'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30',
       'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'filepath', 'index',
       'filename', 'parent', 'gparent', 'bearfile', 'timeinfo', 'prefix',
       'ymdinfo', 'postfix', 'is_kaitei', 'ymd', 'next_month_day1',
       'next_month_day2', 'sha256', 'md5', 'size', 'lastfmodification',
       'filecreation', 'ymdc'],
      dtype='object')

In [70]:
def cx_fill(df, c):
    cf = df.groupby("c2")[c].fillna(method="ffill")
    return cf

In [71]:
%%time
# .assign(c13 = lambda df:df.groupby("c2")["c13"].fillna(method ="ffill"))\
    # .assign(c14 = lambda df:df.groupby("c2")["c14"].fillna(method ="ffill"))\
    # .assign(c15 = lambda df:df.groupby("c2")["c15"].fillna(method ="ffill"))\
    # .assign(c16 = lambda df:df.groupby("c2")["c16"].fillna(method ="ffill"))\
    # .assign(c31 = lambda df:df.groupby("c2")["c31"].fillna(method ="ffill"))\

df_meta = df_whole \
    .loc[:, ["c0", "c2", "c4", "c7", "c8", "c9", "c10","c11",
             "c13", "c14","c15", "c16", "c17",
              "c20", "c21","c22", "c29", "c32", "c36",
             "ymd", "next_month_day1", "next_month_day2", "filename"]].copy()\
    .sort_values(["c2", "ymd","filename"], ascending=[True, True, False]) \
    .assign(c4 = lambda df:cx_fill(df, "c4"))\
    .assign(c7 = lambda df:cx_fill(df, "c7"))\
    .assign(c8 = lambda df:cx_fill(df, "c8"))\
    .assign(c9 = lambda df:cx_fill(df, "c9"))\
    .assign(c10 = lambda df:cx_fill(df, "c10"))\
    .assign(c11 = lambda df:cx_fill(df, "c11"))\
    .assign(c13 = lambda df:cx_fill(df, "c13"))\
    .assign(c14 = lambda df:cx_fill(df, "c14"))\
    .assign(c15 = lambda df:cx_fill(df, "c15"))\
    .assign(c16 = lambda df:cx_fill(df, "c16"))\
    .assign(c17 = lambda df:cx_fill(df, "c17"))\
    .assign(c20 = lambda df:cx_fill(df, "c20"))\
    .assign(c21 = lambda df:cx_fill(df, "c21"))\
    .assign(c22 = lambda df:cx_fill(df, "c22"))\
    .assign(c29 = lambda df:cx_fill(df, "c29"))\
    .assign(c32 = lambda df:cx_fill(df, "c32"))\
    .assign(c36 = lambda df:cx_fill(df, "c36"))\
    .query("c10 == c10")\
    .assign(dt_rank_neg = lambda df:df.groupby("c2")["ymd"].rank(ascending = False,method='first'))\
    .query("dt_rank_neg < 2") \

#     .assign(c4 = lambda df:df.groupby("c2")["c4"].fillna(method ="ffill"))\
#     .assign(c7 = lambda df:df.groupby("c2")["c7"].fillna(method ="ffill"))\
#     .assign(c9 = lambda df:df.groupby("c2")["c9"].fillna(method ="ffill"))\
#     .assign(c10 = lambda df:df.groupby("c2")["c10"].fillna(method ="ffill"))\
#     .assign(c89 = lambda df:df.groupby("c2")["c89"].fillna(method ="ffill"))\
#     .assign(c90 = lambda df:df.groupby("c2")["c90"].fillna(method ="ffill"))\
#     .assign(c91 = lambda df:df.groupby("c2")["c91"].fillna(method ="ffill"))\
#     .assign(c92 = lambda df:df.groupby("c2")["c92"].fillna(method ="ffill"))\
#     .assign(c93 = lambda df:df.groupby("c2")["c93"].fillna(method ="ffill"))\
#     .assign(c94 = lambda df:df.groupby("c2")["c94"].fillna(method ="ffill"))\
#     .assign(c89 = lambda df:df.c89.astype(int))\
#     .assign(c90 = lambda df:df.c90.astype(int))\
#     .assign(c91 = lambda df:df.c91.astype(int))\
#     .assign(c92 = lambda df:df.c92.astype(int))\
#     .assign(c93 = lambda df:df.c93.astype(int))\
#     .assign(c94 = lambda df:df.c94.astype(int))
    
df_meta.head(50)

/tmp/ipykernel_69704/4156245904.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cf = df.groupby("c2")[c].fillna(method="ffill")


CPU times: user 5.84 s, sys: 59.6 ms, total: 5.9 s
Wall time: 5.88 s


,c0,c2,c4,c7,c8,c9,c10,c11,c13,c14,c15,c16,c17,c20,c21,c22,c29,c32,c36,ymd,next_month_day1,next_month_day2,filename,dt_rank_neg
15479,0,700010000,半切,6.0,1.0,枚,1.0,120.0,1.0,0.0,0.0,1.0,120.0,0.0,0.0,0.0,,0.0,フィルム・半切,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
15480,0,700020000,半切（２分割）,6.0,1.0,枚,1.0,120.0,1.0,0.0,0.0,1.0,120.0,0.0,0.0,0.0,,0.0,フィルム・半切（２分割）,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
15481,0,700030000,大角,6.0,1.0,枚,1.0,115.0,1.0,0.0,0.0,1.0,115.0,0.0,0.0,0.0,,0.0,フィルム・大角,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
15482,0,700040000,大角（２分割）,6.0,1.0,枚,1.0,115.0,1.0,0.0,0.0,1.0,115.0,0.0,0.0,0.0,,0.0,フィルム・大角（２分割）,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
15483,0,700050000,大四ツ切,6.0,1.0,枚,1.0,76.0,1.0,0.0,0.0,1.0,76.0,0.0,0.0,0.0,,0.0,フィルム・大四ツ切,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
15484,0,700060000,大四ツ切（２分割）,6.0,1.0,枚,1.0,76.0,1.0,0.0,0.0,1.0,76.0,0.0,0.0,0.0,,0.0,フィルム・大四ツ切（２分割）,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
15485,0,700070000,大四ツ切（４分割）,6.0,1.0,枚,1.0,76.0,1.0,0.0,0.0,1.0,76.0,0.0,0.0,0.0,,0.0,フィルム・大四ツ切（４分割）,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
15486,0,700080000,四ツ切,6.0,1.0,枚,1.0,62.0,1.0,0.0,0.0,1.0,62.0,0.0,0.0,0.0,,0.0,フィルム・四ツ切,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
15487,0,700090000,四ツ切（２分割）,6.0,1.0,枚,1.0,62.0,1.0,0.0,0.0,1.0,62.0,0.0,0.0,0.0,,0.0,フィルム・四ツ切（２分割）,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
15488,0,700100000,四ツ切（４分割）,6.0,1.0,枚,1.0,62.0,1.0,0.0,0.0,1.0,62.0,0.0,0.0,0.0,,0.0,フィルム・四ツ切（４分割）,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0


In [72]:
df_meta\
    .query('c4.str.contains("マイクロ")', engine='python')\
    .head()

,c0,c2,c4,c7,c8,c9,c10,c11,c13,c14,c15,c16,c17,c20,c21,c22,c29,c32,c36,ymd,next_month_day1,next_month_day2,filename,dt_rank_neg
15565,0,710010048,人工内耳用ヘッドセット（マイクロホン）,0.0,0.0,NaN,1.0,38700.0,0.0,0.0,0.0,1.0,38700.0,0.0,0.0,0.0,,0.0,人工内耳用材料・人工内耳用ヘッドセット・マイクロホン,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
16018,0,710010910,血管造影用マイクロカテーテル（遠位端可動型治療用）,0.0,0.0,NaN,1.0,74500.0,0.0,0.0,0.0,1.0,74500.0,0.0,0.0,0.0,,0.0,血管造影用マイクロカテーテル・遠位端可動型治療用,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
8131,9,710010911,血管造影用マイクロカテーテル（遠位端可動型治療用）・指定,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,75900.0,0.0,0.0,0.0,20180331,0.0,血管造影用マイクロカテーテル・遠位端可動型治療用・指定承認番号２２６００ＢＺＸ００４８２０００,2018-04-01,2018-05-01,2018-05-02,t20180305.csv,1.0
16421,0,732970000,血管造影用マイクロカテーテル（アプローチ型・ブレード有）,0.0,0.0,NaN,1.0,36900.0,0.0,0.0,0.0,1.0,37900.0,0.0,0.0,0.0,,0.0,血管造影用マイクロカテーテル・オーバーザワイヤー・選択的アプローチ型・ブレードあり,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0
16422,0,732980000,血管造影用マイクロカテーテル（アプローチ型・ブレード無）,0.0,0.0,NaN,1.0,35800.0,0.0,0.0,0.0,1.0,35800.0,0.0,0.0,0.0,,0.0,血管造影用マイクロカテーテル・オーバーザワイヤー・選択的アプローチ型・ブレードなし,2022-04-01,2022-05-01,2022-05-02,t_ALL20231018.csv,1.0


In [73]:
# df_meta \
#     .query("c2 ==150443250")

In [74]:
# df_sub.query("c2 == 111000110")

In [75]:
df_sub["c2"].unique().size

1553

In [76]:
[c2 for c2 in list(df_sub["c2"].unique()) if c2 not in df_meta["c2"].to_list()]

[]

In [77]:
102 % 100

2

In [92]:
d_output = "../output_to/"
outputtag = "tocode_"
# postfix = ".json.gz"
postfix = ".json"
pf = d_output + outputtag + "{}" + "_" + ts + postfix
pf

'../output_to/tocode_{}_20231122_150209_109733.json'

In [93]:
import json
# import gzip
import bz2
import joblib
# with gzip.open(write_file, 'wt', encoding="ascii") as zipfile:
#        json.dump(data, zipfile)
        
def d2j(d, f_json):
    with open(f_json,  mode='wt', encoding='utf-8') as file:
        json.dump(d, file, ensure_ascii=False, indent=2)

def d2jz(d, f_json):
    # with gzip.open(f_json,  mode='wt', encoding='utf-8') as zipfile:
    with bz2.open(f_json,  mode='wt', encoding='utf-8') as zipfile:
        json.dump(d, zipfile, ensure_ascii=False, indent=2)


いまさらだが書き出しは並列でやったほうが早い。  
当初ベタなループであったが、並列処理に修正。  
コアもりもりの計算機ならすぐ終わるはず。

ctn: code table number:コード表用番号  
マスターカラム90-94  
c89-93  

In [94]:
# c20	c21	c22	c29	c32

In [97]:
def c2j(c2):
    df_iter = dfdt.copy()\
        .loc[:,["dt","dt_str"]]\
        .assign(c2 = c2)\
        .merge(df_span, on= "c2")\
        .query("t1 <= dt and dt < t2")
    
    d_meta_sub = df_meta\
    .query("c2 == @c2").to_dict('records')[0]

    f_out = pf.format(c2)
    # print(f_out)
    d_i = {
        "sicode":int(c2),
        "name":d_meta_sub["c4"],
        "pointlabel":d_meta_sub["c10"],
        "c20":d_meta_sub["c20"],
        "c21":d_meta_sub["c21"],
        "c22":d_meta_sub["c22"],
        "c29":d_meta_sub["c29"],
        "c32":d_meta_sub["c32"],
        "dt_start": dt_start_str,
        "dt_end":dt_end_str,
        "dt":dict()
    }
    
    for r in df_iter.itertuples():
        d_i["dt"][r.dt_str] = {"tag":r.tag,
                              "point":r.point}
    
    # print(d_i)
    d2j(d=d_i, f_json=f_out)
    # d2jz(d=d_i, f_json=f_out)
    
    return f_out

c2list = list(df_sub["c2"].unique())
len(c2list)

1553

In [98]:
%%time
result = joblib.Parallel(n_jobs=-5,verbose=2)(joblib.delayed(c2j)(i) for i in c2list)

[Parallel(n_jobs=-5)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-5)]: Done  33 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-5)]: Done 154 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-5)]: Done 357 tasks      | elapsed:   53.1s
[Parallel(n_jobs=-5)]: Done 640 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-5)]: Done 1005 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-5)]: Done 1450 tasks      | elapsed:  3.7min


CPU times: user 28 s, sys: 2.83 s, total: 30.9 s
Wall time: 3min 55s


[Parallel(n_jobs=-5)]: Done 1553 out of 1553 | elapsed:  3.9min finished


In [99]:
f_meta_x = "../appendix/dfmeta_to_" + ts + ".xlsx"
f_meta_pk = "../appendix/dfmeta_to_" + ts + ".pkl"

df_meta.to_excel(f_meta_x)
df_meta.to_pickle(f_meta_pk)
f_meta_x, f_meta_pk

('../appendix/dfmeta_to_20231122_150209_109733.xlsx',
 '../appendix/dfmeta_to_20231122_150209_109733.pkl')

In [100]:
f_span_x = "../appendix/dfspan_to_" + ts + ".xlsx"
f_span_pk = "../appendix/dfspan_to_" + ts + ".pkl"
df_span.to_excel(f_span_x)
df_span.to_pickle(f_span_pk)
f_span_x, f_span_pk

('../appendix/dfspan_to_20231122_150209_109733.xlsx',
 '../appendix/dfspan_to_20231122_150209_109733.pkl')

In [48]:
df_sub\
    .query("c2 ==150153910")

,c0,c2,c4,c9,c10,c11,c16,c20,c58,c59,c90,c91,c92,c93,c94,c88,c99,ymd,next_month_day1,next_month_day2,filename,c11fill_,c11fill,c59fill_,c59fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master
8615,0,150153910,経皮的冠動脈形成術,NaN,3.0,22000.0,0.0,3.0,0.0,0.0,10.0,546.0,0.0,0.0,0.0,57260000.0,0.0,2014-01-10,2014-02-01,2014-02-02,s_ALL20140110.csv,22000.0,22000.0,0.0,0.0,2010-01-01,2014-02-02,2014-04-02,1.0,True,False
13739,9,150153910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-05,2014-04-01,2014-04-02,s_20140305.csv,22000.0,22000.0,0.0,0.0,2014-02-02,2014-04-02,2023-06-03,2.0,False,True
